# Benchmark Results

For object creation and selecting, using schema objects increases time by 2-3x. This means using object-oriented approach incurs a huge cost over using dictionaries.

In [1]:
import sys
sys.path.append('..')
import doctable
import dataclasses
import functools
import tempfile
import random
import copy

## Basic Benchmark: Schema Object VS Raw Dictionaries

In [3]:
@doctable.schema
class MyObjSmall:
    __slots__ = []
    id: int = doctable.Col()
    name: str = doctable.Col()

@doctable.schema
class MyObjBig:
    __slots__ = []
    id: int = doctable.Col()
    name: str = doctable.Col()
    extra1: str = doctable.Col()
    extra2: str = doctable.Col()
    extra3: str = doctable.Col()
    extra4: str = doctable.Col()
    extra5: str = doctable.Col()

In [11]:

def make_test_dt(SchemaClass, tmpdir) -> doctable.DocTable:
    dt_small = doctable.DocTable(
        schema=SchemaClass,
        target = f'{tmpdir}/{SchemaClass}_test.db',
        new_db = True,
    )
    return dt_small
def runtest_obj(n: int = 100):
    print(f'=== start OBJECT test: {n=} ================')
    with tempfile.TemporaryDirectory() as tmpdir:
        dt_small = make_test_dt(MyObjSmall, tmpdir)
        print(f'construct {n} objects')
        %timeit [MyObjSmall(i, f'name_{i}') for i in range(n)]
        test_objs = [MyObjSmall(i, f'name_{i}') for i in range(n)]
        print(f'insert objects')
        %time dt_small.q.insert_multi(test_objs)
        print(f'select objects')
        %timeit dt_small.q.select()

def runtest_raw(n: int = 100):
    print(f'=== start RAW test: {n=} ================')
    with tempfile.TemporaryDirectory() as tmpdir:
        dt_small = make_test_dt(MyObjSmall, tmpdir)
        print(f'construct {n} objects')
        %timeit [{'id': i, 'name': f'name_{i}'} for i in range(n)]
        test_objs = [{'id': i, 'name': f'name_{i}'} for i in range(n)]
        print(f'insert objects')
        %time dt_small.q.insert_multi_raw(test_objs)
        print(f'select objects')
        %timeit dt_small.q.select_raw()

for n in [100, 1000, 10000, 100000, 1000000]:
    print(f'================== {n=}=============================')
    runtest_obj(n)
    runtest_raw(n)




================== n=100=============================
=== start test: n=100 ================
construct 100 objects
66.6 µs ± 76.5 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
insert objects
CPU times: user 0 ns, sys: 1.65 ms, total: 1.65 ms
Wall time: 3.77 ms
select objects
919 µs ± 5.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
=== start test: n=100 ================
construct 100 objects
21.5 µs ± 100 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
insert objects
CPU times: user 1.5 ms, sys: 0 ns, total: 1.5 ms
Wall time: 8.28 ms
select objects
405 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
================== n=1000=============================
=== start test: n=1000 ================
construct 1000 objects
659 µs ± 454 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
insert objects
CPU times: user 5.24 ms, sys: 0 ns, total: 5.24 ms
Wall time: 8.58 ms
select objects
5.92 ms ± 15.3 µs per loop (mean ± std. dev. of 